In [1]:
%pip install vosk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install sounddevice 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: transformers in c:\users\sredd\appdata\local\programs\python\python311\lib\site-packages (4.47.1)




[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import threading
import sounddevice as sd
import queue
import vosk
from vosk import Model, KaldiRecognizer
from transformers import pipeline 
import json

In [7]:
model = Model(lang="en-us")

In [8]:
recognizer = KaldiRecognizer(model, 16000)

In [9]:
q = queue.Queue()

In [60]:
def callback(indata, frames, time, status):
    q.put(bytes(indata))

In [79]:
stop_event = threading.Event()

In [80]:
def listen_for_stop():
    while True:
        user_input = input()
        if user_input.lower() == 'stop':
            stop_event.set()
            break

In [81]:
# with sd.RawInputStream(samplerate=16000, blocksize=8000, dtype='int16',
#                        channels=1, callback=callback):
#     print("Say something")

#     while True:
#         data = q.get()
#         if recognizer.AcceptWaveform(data):
#             result = recognizer.Result()
#             print(json.loads(result))
#         else:
#             partial_result = recognizer.PartialResult()
#             print(json.loads(partial_result))

#     final_result = recognizer.FinalResult()
#     print(json.loads(final_result))

listener_thread = threading.Thread(target=listen_for_stop)
listener_thread.start()

with sd.RawInputStream(samplerate=16000, blocksize=8000, dtype='int16',
                       channels=1, callback=callback):
    print("Say something")

    while not stop_event.is_set():
        data = q.get()
        if recognizer.AcceptWaveform(data):
            result = recognizer.Result()
            # print(json.loads(result))
        else:
            partial_result = recognizer.PartialResult()
            # print(json.loads(partial_result))

    final_result = recognizer.FinalResult()
    print(json.loads(final_result))

Say something
{'text': 'hello how are you'}


In [26]:
ner_pipeline = pipeline("ner", model="d4data/biomedical-ner-all", aggregation_strategy="simple")

c:\Users\sredd\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sredd\.cache\huggingface\hub\models--d4data--biomedical-ner-all. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [27]:
def redact_phi(text):
    entities = ner_pipeline(text)
    redacted_text = text
    for entity in sorted(entities, key=lambda x: x['start'], reverse=True):
        redacted_text = redacted_text[:entity['start']] + f"[{entity['entity_group'].upper()}_REDACTED]" + redacted_text[entity['end']:]
    return redacted_text

In [52]:
text = (json.loads(final_result))['text']
print(text)
ner_transcript = redact_phi(text)
print(ner_transcript)

[DIAGNOSTIC_PROCEDURE_REDACTED] [DIAGNOSTIC_PROCEDURE_REDACTED] [NONBIOLOGICAL_LOCATION_REDACTED]
